# Misc code - discrete heterogeneous reconstruction and symmetry

Not working - just a bunch of code snippets that used to be in the ```Reconstruction_data``` notebook but were not used. Moving them here in case I get back to this code later.

Also including at the end some unrelated bits of code from notebooks, not used at the moment but might be useful later.

In [1]:
import time
import mrcfile
import jax
import jax.numpy as jnp
from jax import random
from jax.config import config
from jax.scipy.special import gammaln
import numpy as np
from matplotlib import pyplot as plt

from simplecryoem.datasets import create_het_dataset
from simplecryoem.emfiles import load_data
from simplecryoem.utils import (
    plot_angles,
    volume_fourier,
    crop_fourier_volume,
    mip_y,
    mip_z,
    create_3d_mask,
)
from simplecryoem.mcmc import accept_reject_vmap
from simplecryoem.projection import project
from simplecryoem.mcmc import mcmc

In [2]:
%load_ext autoreload
%autoreload 2

plt.rcParams["figure.figsize"] = 6, 6
config.update("jax_enable_x64", True)

In [3]:
# Out directory to save outputs
out_dir = "../data/out_dev/"

In [4]:
# with open(f"{data_dir}/z.pkl", 'rb') as file:
#    # Call load method to deserialze
#    z = pickle.load(file)

#### Or a custom heterogeneous dataset with 2 classes

In [ ]:
nx = 64
N_set = [5000, 5000]

data_dir1 = "/project/lederman/rrl28/share/CryoEMDataSmall/RelionSample/relion30_tutorial_precalculated_results/"
star_file1 = "Extract/job020/particles.star"

data_dir2 = "/gpfs/gibbs/pi/lederman/dge5/CrossValidationCryodrgnSet/"
star_file2 = "particles_subset_5k.star"

data_dirs = [data_dir1, data_dir2]
star_files = [star_file1, star_file2]

imgs0, imgs_f, ctf_params, pixel_size, angles, shifts, z, x_grid = create_het_dataset(
    data_dirs, star_files, N_set, nx
)

N = imgs_f.shape[0]

In [ ]:
# plt.plot(z, '.')
nbins = 100
counts, bins = np.histogram(z, bins=nbins)
_ = plt.hist(bins[:-1], bins, weights=counts)

In [ ]:
plt.imshow(imgs0[1][3])
plt.colorbar()

In [ ]:
plot_angles(angles[np.random.permutation(imgs_f.shape[0])[:2000]])

In [ ]:
idx = 2
plt.imshow(np.abs(jnp.fft.fftshift(imgs_f[idx].reshape([nx, nx]))))
plt.colorbar()
print(f"z[{idx}] = {z[idx]}")

In [ ]:
idx = 1037
plt.imshow(np.abs(jnp.fft.fftshift(imgs_f[idx].reshape([nx, nx]))))
plt.colorbar()
print(f"z[{idx}] = {z[idx]}")

#### Or read 3 classes from Daniel's simulated data (TODO: move this to the ```datasets.py``` file)

In [ ]:
classes = [0, 4, 9]

data_dir = "/gpfs/gibbs/pi/lederman/dge5/SimulatedData/"
star_file = f"projections/volume{classes[0]}.star"

params0, imgs0 = load_data(data_dir, star_file, load_imgs=True, fourier=False)


data_dir = "/gpfs/gibbs/pi/lederman/dge5/SimulatedData/"
star_file = f"projections/volume{classes[1]}.star"

params1, imgs1 = load_data(data_dir, star_file, load_imgs=True, fourier=False)

data_dir = "/gpfs/gibbs/pi/lederman/dge5/SimulatedData/"
star_file = f"projections/volume{classes[2]}.star"

params2, imgs2 = load_data(data_dir, star_file, load_imgs=True, fourier=False)

imgs0 = np.concatenate([imgs0, imgs1, imgs2], axis=0)

ctf_params0 = np.concatenate(
    [params0["ctf_params"], params1["ctf_params"], params2["ctf_params"]], axis=0
)
pixel_size0 = np.concatenate(
    [params0["pixel_size"], params1["pixel_size"], params2["pixel_size"]], axis=0
)
angles0 = np.concatenate(
    [params0["angles"], params1["angles"], params2["angles"]], axis=0
)
shifts0 = np.concatenate(
    [params0["shifts"], params1["shifts"], params2["shifts"]], axis=0
)

# Assuming all classes have the same number of images
Nk = imgs1.shape[0]
z_true0 = jnp.zeros((Nk,))
z_true1 = jnp.ones((Nk,))
z_true2 = 2 * jnp.ones((Nk,))

z_true = jnp.concatenate([z_true0, z_true1, z_true2])

del (imgs1, imgs2, params0, params1, params2)

In [ ]:
# z_true = z
# z_true = z_true[idxrand]

# nbins = 100
# counts, bins = np.histogram(z_true, bins=nbins)
# _ = plt.hist(bins[:-1], bins, weights=counts)

### Class sampling 

In [ ]:
# Load the correct volumes

print(classes)

vols = []
vols_r = []
for i in classes:
    with mrcfile.open(
        f"/gpfs/gibbs/pi/lederman/dge5/SimulatedData/{i}_volume.mrc"
    ) as file:
        v = file.data

    v_f, _, _ = volume_fourier(v, pixel_size[0])
    v_f, _ = crop_fourier_volume(v_f, x_grid, 16)

    v_r = jnp.fft.fftshift(jnp.real(jnp.fft.ifftn(v_f)))

    vols.append(v_f)
    vols_r.append(v_r)

vols = jnp.array(vols)
vols_r = jnp.array(vols_r)
# Crop them

In [ ]:
# v_f = np.fft.fftshift(v_f)

mip_z((jnp.abs(vols_r[0])))
plt.colorbar()

In [ ]:
plt.imshow(jnp.abs((v_f[0])))
plt.colorbar()

In [ ]:
mip_z(vols_r[0])

In [ ]:
def get_class_proposal_func(loss_func_batched, loss_func_sum, sigma_noise, alpha_d, K):
    def logPi(v, angles, shifts, ctf_params, imgs, z, sigma_noise):
        n_k = calc_nk_jit(z)
        term1 = -loss_func_sum(v, angles, shifts, ctf_params, imgs, z, sigma_noise)

        logPZalpha = jnp.sum(gammaln(n_k + alpha_d))

        return term1 + logPZalpha

    calc_nk_k = lambda z, k: jnp.sum(z == k)
    calc_nk = lambda z: jax.vmap(calc_nk_k, in_axes=(None, 0))(z, jnp.arange(K))
    calc_nk_jit = jax.jit(calc_nk)

    @jax.jit
    def proposal_z_idx(key, z0, logPiX0, v, angles, shifts, ctf_params, imgs, idx):
        logPi_local = lambda z: logPi(
            v, angles, shifts, ctf_params, imgs, z, sigma_noise
        )

        newval = random.randint(key, (), 0, K)
        z1 = z0.at[idx].set(newval)

        logPiX0 = logPi_local(z0)
        logPiX1 = logPi_local(z1)

        r = jnp.exp(logPiX1 - logPiX0)

        return z1, r, logPiX1, logPiX0

    @jax.jit
    def proposal_z_batch_old(key, z0, logPiX0, v, angles, shifts, ctf_params, imgs):
        """Batch mode proposal function for z, operating in the same
        way as proposal_x_batch.

        If the posterior depends on the states z_1,...,z_N (e.g. due to
        the Dirichlet prior), then running MCMC with this proposal function
        is NOT correct. However, this function is used in the
        proposal_z_batch_correct" function, which takes advantage of these
        parallel proposals in a proper MCMC way."""

        N = angles.shape[0]
        keys = random.split(key, N)

        # TODO: change both batch functions (this and proposal_x_batch)
        # so we don't have to do diag on an N x N (or K x K) matrix - potentially
        # problematic for large N.
        z1, r, logPiX1, logPiX0 = jax.vmap(
            proposal_z_idx, in_axes=(0, None, None, None, None, None, None, None, 0)
        )(keys, z0, logPiX0, v, angles, shifts, ctf_params, imgs, jnp.arange(N))
        z1 = jnp.diag(z1)

        return z1, r, logPiX1, logPiX0

    @jax.jit
    def proposal_z_batch(key, z0, logPiX0, v, angles, shifts, ctf_params, imgs):
        """Batch mode proposal function for z, with no Dirichlet prior
        for the classes.

        If the posterior depends on the states z_1,...,z_N (e.g. due to
        the Dirichlet prior), then running MCMC with this proposal function
        is NOT correct. However, this function is used in the
        proposal_z_batch_correct" function, which takes advantage of these
        parallel proposals in a proper MCMC way."""

        N = angles.shape[0]
        z1 = random.randint(key, z0.shape, 0, K)

        logPiX0 = -loss_func_batched(
            v, angles, shifts, ctf_params, imgs, z0, sigma_noise
        )
        logPiX1 = -loss_func_batched(
            v, angles, shifts, ctf_params, imgs, z1, sigma_noise
        )
        r = jnp.exp(logPiX1 - logPiX0)

        return z1, r, logPiX1, logPiX0

    @jax.jit
    def proposal_z_batch_correct(key, z0, logPiX0, v, angles, shifts, ctf_params, imgs):
        """If using the Dirichlet posterior, then the proposal_z_batch is not
        proper MCMC sampling. Here, we run a number of "incorect" MCMC steps to
        sample all the entries of z in parallel, and then use the latest
        proposed z as a proposal for a "correct" MCMC proposal. The acceptance rate
        is pretty good in this toy example and the convergence great."""

        logPi_local = lambda z: logPi(
            v, angles, shifts, ctf_params, imgs, z, sigma_noise
        )

        N = angles.shape[0]
        N_samples_z_local = 10
        keys = random.split(key, 2 * N_samples_z_local)
        params_z = {
            "v": v,
            "angles": angles,
            "shifts": shifts,
            "ctf_params": ctf_params,
            "imgs": imgs,
        }

        logPiZ0 = logPi_local(z0)

        # logPiX0 = logPiZ0
        # for i in jnp.arange(N_samples_z_local):
        #    z1, r, logPiX1, logPiX0 = proposal_z_batch(keys[2*i], z0, logPiX0, **params_z)
        #    a = jnp.minimum(1, r)
        #
        #    unif_var = random.uniform(keys[2*i+1], (N,))
        #    z1, logPiX1 = accept_reject_vmap(unif_var, a, z0, z1, logPiX0, logPiX1)
        #
        #    z0 = z1
        #    logPiX0 = logPiX1

        def body_fun(i, z0logPiX0):
            z0, logPiX0 = z0logPiX0

            z1, r, logPiX1, logPiX0 = proposal_z_batch(
                keys[2 * i], z0, logPiX0, **params_z
            )
            a = jnp.minimum(1, r)

            unif_var = random.uniform(keys[2 * i + 1], (N,))
            z1, logPiX1 = accept_reject_vmap(unif_var, a, z0, z1, logPiX0, logPiX1)

            return z1, logPiX1

        z1, _ = jax.lax.fori_loop(
            0, N_samples_z_local, body_fun, (z0, logPiZ0 * jnp.ones((N,)))
        )

        logPiZ1 = logPi_local(z1)
        r = jnp.exp(logPiZ1 - logPiZ0)

        return z1, r, logPiZ1, logPiZ0

    return proposal_z_idx, proposal_z_batch, proposal_z_batch_correct

In [ ]:
K = len(classes)
alpha_d = 10 * jnp.ones(K)

alpha = 1e-9
# alpha = 1e-6
# alpha=1e-7

# radius = 0.048# for nx=32
radius = 0.09
# radius = 0.185
# radius = 1.6 # for nx=64
# radius = 0.198 # for nx=128)
mask = create_3d_mask(x_grid, (0, 0, 0), radius)
mip_z(jnp.fft.fftshift(mask))
plt.colorbar()
M = 1 / jnp.max(sigma_noise) ** 2 * jnp.ones([nx, nx, nx])
dt_list = jnp.array([0.1, 0.5, 1, 5, 10])
L = 5

pixel_size_nx = pixel_size[0] * 320 / nx
B = pixel_size_nx * nx / 15
B_list = jnp.array([B, B / 2, B / 4, B / 8])

slice_func, _, _ = get_slice_funcs(project, x_grid, mask, "tri")

loss_func, loss_func_batched, loss_func_sum = get_loss_funcs(slice_func, alpha=alpha)
_, loss_func_batched0, _ = get_loss_funcs(slice_func, alpha=0)

grad_loss_volume, grad_loss_volume_sum = get_grad_v_funcs(loss_func, loss_func_sum)

loss_proj_func_batched0 = get_loss_proj_funcs(apply_shifts_and_ctf, x_grid, alpha=0)


# logPi = lambda v : -loss_func_sum(v, angles_rec, shifts_rec, ctf_params, imgs_f, sigma_noise)
# gradLogPi = lambda v : -jnp.conj(grad_loss_volume_sum(v, angles_rec, shifts_rec, ctf_params, imgs_f, sigma_noise))


(
    loss_func_z,
    loss_func_z_batched,
    loss_func_z_sum,
    grad_loss_volume_z,
    grad_loss_volume_sum_z,
) = get_loss_grad_funcs_classes(loss_func)

proposal_z_idx, proposal_z_batch, proposal_z_batch_correct = get_class_proposal_func(
    loss_func_z_batched, loss_func_z_sum, sigma_noise, alpha_d, K
)

(
    proposal_func_orientations_u,
    proposal_func_orientations_p,
    proposal_func_shifts,
    proposal_func_vol,
    proposal_func_vol_batch,
) = get_jax_proposal_funcs(
    loss_func_z_batched,
    loss_proj_func_batched0,
    loss_func_z_sum,
    grad_loss_volume_sum_z,
    sigma_noise,
    B_list,
    dt_list,
    L,
    M,
)

In [ ]:
Nn = N

N_iters = 10
N_samples_z = 201
N_samples_v = 201
N_samples_a = 1001
key = random.PRNGKey(int(jnp.floor(np.random.rand() * 1000)))

sample_v = False
sample_z = True
sample_a = False

# v0 = jnp.array(np.random.randn(K, nx,nx,nx) + 1j * np.random.randn(K, nx,nx,nx))
v0 = vols.astype(jnp.complex128)

z0 = jnp.array(np.random.randint(0, K, (Nn,)))

# a0 = generate_uniform_orientations_jax(key, angles)
a0 = angles[:Nn]


vi = v0
zi = z0
ai = a0

t0 = time.time()

for iter_i in jnp.arange(N_iters):
    print(f"Iter = {iter_i}")

    if sample_v:
        print("  Sampling vols")

        params_v = {
            "angles": ai[:Nn],
            "shifts": shifts[:Nn],
            "ctf_params": ctf_params[:Nn],
            "imgs_iter": imgs_f[:Nn],
            "z": zi[:Nn],
        }

        key, subkey = random.split(key)

        v_mean, v_r, v_samples = mcmc(
            subkey, proposal_func_vol, vi, N_samples_v, params_v, save_samples=-1
        )

        vi = v_samples[N_samples_v - 2]

    if sample_z:
        print("  Sampling z")

        params_z = {
            "v": vi,
            "angles": ai[:Nn],
            "shifts": shifts[:Nn],
            "ctf_params": ctf_params[:Nn],
            "imgs": imgs_f[:Nn]
            # 'idx' : idx
        }

        key, subkey = random.split(key)

        # v_hmc_mean, r_hmc, v_hmc_samples = mcmc(subkey, proposal_func_vol_batch, v0, N_mc_samples, params_vol, save_samples=-1)

        # _, z_r, z_samples = mcmc(subkey, proposal_z_batch, zi, N_samples_z, params_z, Nn, save_samples=1)
        _, z_r, z_samples = mcmc(
            subkey,
            proposal_z_batch_correct,
            zi,
            N_samples_z,
            params_z,
            1,
            save_samples=1,
        )
        zi = z_samples[N_samples_z - 2]

    if sample_a:
        print("  Sampling orientations")

        params_a = {
            "v": vi,
            "shifts": shifts[:Nn],
            "ctf_params": ctf_params[:Nn],
            "imgs_iter": imgs_f[:Nn],
            "z": zi[:Nn],
        }

        key, subkey = random.split(key)

        _, a_r, a_samples = mcmc(
            key,
            proposal_func_orientations_u,
            ai,
            N_samples_a,
            params_a,
            Nn,
            1,
            verbose=True,
        )
        ai = a_samples[N_samples_a - 2]


v_rec = vi
print(time.time() - t0)

In [ ]:
v_rec_r = jnp.array([jnp.fft.fftshift(jnp.real(jnp.fft.ifftn(v))) for v in v_rec])

In [ ]:
idx = 4

# plt.plot(z_r[:,idx])
plt.plot(z_r)

In [ ]:
plt.plot(z_samples[:, 1213])

In [ ]:
z_samples.shape

In [ ]:
nbins = 100
counts, bins = np.histogram(z_samples[:, idx], bins=nbins)
_ = plt.hist(bins[:-1], bins, weights=counts)

v_idx = jnp.bincount(z_samples[:, idx]).argmax()

In [ ]:
plt.imshow(jnp.real(jnp.fft.fftshift(jnp.fft.ifftn(imgs_f[idx].reshape([nx, nx])))))

In [ ]:
# v_idx = 0

plt.rcParams["figure.figsize"] = 12, 6
plt.subplot(2, 2, 1)
mip_y(vols_r[v_idx])
plt.colorbar()
plt.title("Ground truth")

plt.subplot(2, 2, 2)
mip_y(v_rec_r[v_idx])
plt.colorbar()
plt.title("Reconstruction")

plt.subplot(2, 2, 3)
mip_y(jnp.real(jnp.fft.fftshift(jnp.fft.ifftn(v0[v_idx]))))
plt.colorbar()
plt.title("Initialization")

plt.rcParams["figure.figsize"] = 6, 6

In [ ]:
mip_x(v_rec_r[0])
plt.colorbar()

In [ ]:
# for i in range(K):
#    with mrcfile.new(f"{out_dir}/class{i}_Dir.mrc", overwrite=True) as mrc:
#        mrc.set_data(v_rec_r[i].astype(np.float32))

In [ ]:
plt.rcParams["figure.figsize"] = 12, 6
plt.subplot(1, 2, 1)
sl = slice_func(vols[v_idx], angles[idx], shifts[idx], ctf_params[idx])
plt.imshow(jnp.real(jnp.fft.fftshift(jnp.fft.ifftn(sl.reshape([nx, nx])))))

plt.subplot(1, 2, 2)
sl = slice_func(v_rec[v_idx], angles[idx], shifts[idx], ctf_params[idx])
plt.imshow(jnp.real(jnp.fft.fftshift(jnp.fft.ifftn(sl.reshape([nx, nx])))))

plt.rcParams["figure.figsize"] = 6, 6

In [ ]:
print(f"v_idx = {v_idx}")
print(f"z_true[{idx}] = {z_true[idx]}")
assert z_true[idx] == v_idx, f"{z_true[idx]} != {v_idx} !!!"

In [ ]:
plt.plot(z0 - z_true[:Nn])

In [ ]:
plt.plot(zi - z_true[:Nn])

In [ ]:
print(jnp.sum(jnp.abs(z0 - z_true[:Nn])) / len(z_true))
print(jnp.sum(jnp.abs(zi - z_true[:Nn])) / len(z_true))

### HMC orientations

In [ ]:
def get_grad_angles_funcs(loss_func):
    @jax.jit
    def grad_loss_angles(v, angles, shifts, ctf_params, img, sigma):
        return jax.grad(loss_func, argnums=1)(v, angles, shifts, ctf_params, img, sigma)

    @jax.jit
    def grad_loss_angles_batched(v, angles, shifts, ctf_params, imgs, sigma):
        return jax.vmap(grad_loss_angles, in_axes=(None, 0, 0, 0, 0, None))(
            v, angles, shifts, ctf_params, imgs, sigma
        )

    return grad_loss_angles, grad_loss_angles_batched

In [ ]:
grad_loss_angles, grad_loss_angles_batched = get_grad_angles_funcs(loss_func)

In [ ]:
print(v_hmc_mean.shape)
print(angles.shape)
print(shifts.shape)
print(ctf_params.shape)
print(imgs_f.shape)

In [ ]:
grad_ang = grad_loss_angles(
    v_hmc_mean, angles[0], shifts[0], ctf_params[0], imgs_f[0], sigma_noise
)

print(grad_ang.shape)
print(grad_ang)

In [ ]:
N = 1000

t0 = time.time()
grads_ang = grad_loss_angles_batched(
    v_hmc_mean, angles[:N], shifts[:N], ctf_params[:N], imgs_f[:N], sigma_noise
)
print(f"{time.time()-t0} seconds")

print(grads_ang.shape)
# print(grads_ang)

### C3 Symmetry

In [ ]:
def split(arr):
    N = arr.shape[0]
    N2 = jnp.floor(N / 2).astype(jnp.int32)

    arr_train = arr[:N2]
    arr_test = arr[N2:]

    return arr_train, arr_test


def split_and_triple(arr):
    N = arr.shape[0]
    N2 = jnp.floor(N / 2).astype(jnp.int32)

    arr_train = jnp.tile(arr[:N2], reps=(3, 1))
    arr_test = jnp.tile(arr[N2:], reps=(3, 1))

    return arr_train, arr_test


def split_and_triple_angles(a):
    N = a.shape[0]
    N2 = jnp.floor(N / 2).astype(jnp.int32)

    a_train1 = a[:N2]
    a_train2 = a_train1.copy()
    a_train3 = a_train1.copy()

    a_train2[:, 2] += 2 * jnp.pi / 3
    a_train3[:, 2] += 4 * jnp.pi / 3

    a_train = jnp.concatenate([a_train1, a_train2, a_train3], axis=0)

    a_test1 = a[N2:]
    a_test2 = a_test1.copy()
    a_test3 = a_test1.copy()

    a_test2[:, 2] += 2 * jnp.pi / 3
    a_test3[:, 2] += 4 * jnp.pi / 3

    a_test = jnp.concatenate([a_test1, a_test2, a_test3], axis=0)

    return a_train, a_test

In [ ]:
imgs_train, imgs_test = split_and_triple(imgs_f)
shifts_train, shifts_test = split_and_triple(shifts)
ctf_params_train, ctf_params_test = split_and_triple(ctf_params)
angles_train, angles_test = split_and_triple_angles(angles)
N = imgs_train.shape[0]

# plot_angles(angles[:1000])

In [ ]:
# imgs_train, imgs_test = split(imgs_f)
# shifts_train, shifts_test = split(shifts)
# ctf_params_train, ctf_params_test = split(ctf_params)
# angles_train, angles_test = split(angles)
# N = imgs_train.shape[0]
# plot_angles(angles_train)